# DATA INGESTION

In [1]:
import os 

In [2]:
%pwd

"c:\\Users\\soulo\\MACHINE_LEARNING_UNIVERSE\\PROJECT'S\\Chest-Cancer-Classification-Using-mlflow-and-DVC\\RESEARCH"

In [3]:
os.chdir('../')
%pwd

"c:\\Users\\soulo\\MACHINE_LEARNING_UNIVERSE\\PROJECT'S\\Chest-Cancer-Classification-Using-mlflow-and-DVC"

# Workflows
### -> Update config.yaml
### -> Update secrets.yaml [Optional]
### -> Update params.yaml
### -> Update the entity
### -> Update the configuration manager in src config
### -> Update the components
### -> Update the pipeline
### -> Update the main.py
### -> Update the dvc.yaml

In [4]:
# 1 Update config.yaml file [done]

## Update entity

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

## Update Configuration manger

In [18]:
from Chest_Cancer_Classification.constants import *
from Chest_Cancer_Classification.utils.common import read_yaml, create_directories

In [28]:
class ConfigurationManger:
    def __init__(self, config_file_path = CONFIG_FILE_PATH , params_file_path = PARAMS_FILE_PATH):
        self.config  = read_yaml(config_file_path)
        self.params  = read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config
    

## Update components

In [22]:
import os
import gdown
import zipfile
import logging
from Chest_Cancer_Classification import logger
from Chest_Cancer_Classification.utils.common import get_size

In [30]:
class DataIngestion:
    def __init__(self , config: DataIngestionConfig):
        self.config = config
        
    def download_file(self) -> str:
        '''
        Fetch the dataset from the google drive (url)
        '''
        try:
            dataset_url = self.config.source_url
            zip_dir = self.config.local_data_file
            os.makedirs('artifacts/data_ingestion' , exist_ok=True)
            logger.info(f'msg=Downloading data from {dataset_url} into file {zip_dir} (zip file)')
            
            file_id = dataset_url.split('/')[-2]
            prefix = 'https://drive.google.com/u/0/uc?/export=download&id='
            
            
            gdown.download(prefix+file_id, zip_dir)
            
            logger.info(f'Downloaded data from {dataset_url} into file {zip_dir} (zip file)')
            
        except Exception as e:
            raise e
        
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            

## Pipeline

In [31]:
try:
    config = ConfigurationManger()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e :
    raise e
    

[2023-12-02 22:31:17,602: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-02 22:31:17,605: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-02 22:31:17,606: INFO: common: created directory at: artifacts]
[2023-12-02 22:31:17,608: INFO: common: created directory at: artifacts/data_ingestion]
[2023-12-02 22:31:17,609: INFO: 4283021077: msg=Downloading data from https://drive.google.com/file/d/1y9ftEl30hdWn61l1JJy5IxOUsp956Zpq/view?usp=sharing into file artifacts/data_ingestion/data.zip (zip file)]


Downloading...
From (uriginal): https://drive.google.com/u/0/uc?/export=download&id=1y9ftEl30hdWn61l1JJy5IxOUsp956Zpq
From (redirected): https://drive.google.com/uc?/export=download&id=1y9ftEl30hdWn61l1JJy5IxOUsp956Zpq&confirm=t&uuid=7e7bf27b-48b4-43eb-b430-9e087a1a93e8
To: c:\Users\soulo\MACHINE_LEARNING_UNIVERSE\PROJECT'S\Chest-Cancer-Classification-Using-mlflow-and-DVC\artifacts\data_ingestion\data.zip
100%|██████████| 49.0M/49.0M [00:04<00:00, 10.7MB/s]


[2023-12-02 22:31:23,707: INFO: 4283021077: Downloaded data from https://drive.google.com/file/d/1y9ftEl30hdWn61l1JJy5IxOUsp956Zpq/view?usp=sharing into file artifacts/data_ingestion/data.zip (zip file)]
